##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 🚀 Agent Tool Patterns and Best Practices

**Welcome to Day-2 of the Kaggle 5-day Agents course!**

In the previous notebook, you learned how to add custom Python functions as tools to your agent. In this notebook, we'll take the next step: **consuming external MCP services** and handling **long-running operations**.

In this notebook, you'll learn how to:

- ✅ **Connect to external MCP servers**
- ✅ **Implement long-running operations** that can pause agent execution for external input
- ✅ **Build resumable workflows** that maintain state across conversation breaks
- ✅ Understand when and how to use these patterns

## ‼️ Please Read

> ❌ **ℹ️ Note: No submission required!**
> This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

> ⏸️ **Note:**  When you first start the notebook via running a cell you might see a banner in the notebook header that reads **"Waiting for the next available notebook"**. The queue should drop rapidly; however, during peak bursts you might have to wait a few minutes.

> ❌ **Note:** Avoid using the **Run all** cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order - one at a time. [See FAQ on 429 errors for more information.](https://www.kaggle.com/code/kaggle5daysofai/day-0-troubleshooting-and-faqs)

**For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.**

## 📖 Get started with Kaggle Notebooks

If this is your first time using Kaggle Notebooks, welcome! You can learn more about using Kaggle Notebooks [in the documentation](https://www.kaggle.com/docs/notebooks).

Here's how to get started:

**1. Verify Your Account (Required)**

To use the Kaggle Notebooks in this course, you'll need to verify your account with a phone number.

You can do this in your [Kaggle settings](https://www.kaggle.com/settings).

**2. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

This creates a private copy of the notebook just for you.

**3. Run Code Cells**

Once you have your copy, you can run code.

Click the ▶️ Run button next to any code cell to execute it.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

Run the cells in order from top to bottom.

**4. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.

---
## ⚙️ Section 1: Setup

### 1.1: Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

### 1.2: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to access the `GOOGLE_API_KEY` you just saved and set it as an environment variable for the notebook to use:

In [70]:
!pip install --upgrade pip
!pip install python-dotenv
!pip install google-adk

In [71]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit. This keeps your code organized and ensures we have access to the necessary building blocks.

In [72]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [73]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---
## 🧰 Section 2: Model Context Protocol

So far, you have learned how to create custom functions for your agents. But connecting to external systems (GitHub, databases, Slack) requires writing and maintaining API clients.

**Model Context Protocol (MCP)** is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

MCP enables agents to:

✅ **Access live, external data** from databases, APIs, and services without custom integration code  
✅ **Leverage community-built tools** with standardized interfaces  
✅ **Scale capabilities** by connecting to multiple specialized servers

### 2.1: How MCP Works

MCP connects your agent (the **client**) to external **MCP servers** that provide tools:

- **MCP Server**: Provides specific tools (like image generation, database access)
- **MCP Client**: Your agent that uses those tools
- **All servers work the same way** - standardized interface

**Architecture:**
```
┌──────────────────┐
│   Your Agent     │
│   (MCP Client)   │
└────────┬─────────┘
         │
         │ Standard MCP Protocol
         │
    ┌────┴────┬────────┬────────┐
    │         │        │        │
    ▼         ▼        ▼        ▼
┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
│ GitHub │ │Slack│ │ Maps │ │ ... │
│ Server │ │ MCP │ │ MCP  │ │     │
└────────┘ └─────┘ └──────┘ └─────┘
```

### 2.2: Using MCP with Your Agent

The workflow is simple:

1. Choose an MCP Server and tool
2. Create the MCP Toolset (configure connection)
3. Add it to your agent
4. Run and test the agent

**Step 1: Choose MCP Server**

For this demo, we'll use the **[Everything MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/everything)** - an npm package (`@modelcontextprotocol/server-everything`) designed for testing MCP integrations.

It provides a `getTinyImage` tool that returns a simple test image (16x16 pixels, Base64-encoded). **Find more servers:** [modelcontextprotocol.io/examples](https://modelcontextprotocol.io/examples)

**‼️ NOTE: This is a demo server to learn MCP.** In production, you'll use servers for Google Maps, Slack, Discord, etc.

**Step 2: Create the MCP Toolset**

The `McpToolset` is used to integrate an ADK Agent with an MCP Server.

**What the code does:**
- Uses `npx` (Node package runner) to run the MCP server
- Connects to `@modelcontextprotocol/server-everything`
- Filters to only use the `getTinyImage` tool (the server has others, but we only need this one)

In [74]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


#### **Behind the scenes:**
1. **Server Launch**: ADK runs `npx -y @modelcontextprotocol/server-everything`
2. **Handshake**: Establishes stdio communication channel
3. **Tool Discovery**: Server tells ADK: "I provide getTinyImage" functionality
4. **Integration**: Tools appear in agent's tool list automatically
5. **Execution**: When agent calls `getTinyImage()`, ADK forwards to MCP server
6. **Response**: Server result is returned to agent seamlessly

**Why This Matters**: You get instant access to tools without writing integration code!

**Step 3: Add MCP tool to agent**

Let's add the `mcp_server` to the agent's tool array and update the agent's instructions to handle requests to generate tiny images.

In [75]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

Create the runner:

In [76]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

**Step 4: Test the agent**

Ask the agent to generate an image. Watch it use the MCP tool:

In [77]:
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Created new session: debug_session_id

User > Provide a sample tiny image


/usr/local/Cellar/ipython/8.5.0/libexec/lib/python3.10/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image:

image_agent > This is a tiny image:



**Display the image:**

The server returns base64-encoded image data. Let's decode and display it:

In [78]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

### 2.3: Extending to Other MCP Servers

The same pattern works for any MCP server - only the `connection_params` change. Here are some examples:

#### **👉 Kaggle MCP Server** - For dataset and notebook operations

Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competitions.

**Connection example:**
```python
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)
```

**What it provides:**
- 📊 Search and download Kaggle datasets
- 📓 Access notebook metadata
- 🏆 Query competition information etc.,

**Learn more:** [Kaggle MCP Documentation](https://www.kaggle.com/docs/mcp)

### 4.1: ⚠️ The Critical Part - Handling Events in Your Workflow

The agent won't automatically handle pause/resume. **Every long-running operation workflow requires you to:**

1. **Detect the pause:** Check if events contain `adk_request_confirmation`
2. **Get human decision:** In production, show UI and wait for user click. Here, we simulate it.
3. **Resume the agent:** Send the decision back with the saved `invocation_id`

### 4.2 Understand Key Technical Concepts

👉 **`events`** - ADK creates events as the agent executes. Tool calls, model responses, function results - all become events

👉 **`adk_request_confirmation` event** - This event is special - it signals "pause here!"
- Automatically created by ADK when your tool calls `request_confirmation()`
- Contains the `invocation_id`
- Your workflow must detect this event to know the agent paused

👉 **`invocation_id`** - Every call to `run_async()` gets a unique `invocation_id` (like "abc123")
- When a tool pauses, you save this ID
- When resuming, pass the same ID so ADK knows which execution to continue
- Without it, ADK would start a NEW execution instead of resuming the paused one

### 4.3: Helper Functions to Process Events

These handle the event iteration logic for you.

**`check_for_approval()`** - Detects if the agent paused
- Loops through all events and looks for the special `adk_request_confirmation` event
- Returns `approval_id` (identifies this specific request) and `invocation_id` (identifies which execution to resume)
- Returns `None` if no pause detected

In [79]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

**`print_agent_response()`** - Displays agent text
- Simple helper to extract and print text from events

In [80]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

**`create_approval_response()`** - Formats the human decision
- Takes the approval info and boolean decision (True/False) from the human
- Creates a `FunctionResponse` that ADK understands
- Wraps it in a `Content` object to send back to the agent

In [81]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


### 4.4: The Workflow Function - Let's tie it all together!

The `run_shipping_workflow()` function orchestrates the entire approval flow.

Look for the code explanation in the cell below.

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/lro-workflow.png" width="1000" alt="Long-running operation workflow">

#### **Code breakdown**

**Step 1: Send initial request to the Agent**
- Call `run_async()` to start agent execution
- Collect all events in a list for inspection

**Step 2: Detect Pause**
- Call `check_for_approval(events)` to look for the special event: `adk_request_confirmation`
- Returns approval info (with `invocation_id`) if the special event is present; `None` if completed

**Step 3: Resume execution**

PATH A:
- If the approval info is present, at this point the Agent *pauses* for human input.
- Once the Human input is available, call the agent again using `run_async()` and pass in the Human input.
- **Critical:** Same `invocation_id` (tells ADK to RESUME, not restart)
- Display agent's final response after resuming

PATH B:
- If the approval info is not present, then approval is not needed and the agent completes execution.

---

## 📊 Section 6: Summary - Key Patterns for Advanced Tools

In this notebook, you implemented two powerful, production-ready patterns for extending your agent's capabilities beyond simple functions.

| Pattern | When to Use It | Key ADK Components |
| :--- | :--- | :--- |
| **MCP Integration** | You need to connect to **external, standardized services** (like time, databases, or file systems) without writing custom integration code. | `McpToolset` |
| **Long-Running Operations** | You need to **pause a workflow** to wait for an external event, most commonly for **human-in-the-loop** approvals or long background tasks or for compliance/security checkpoints. | `ToolContext`, `request_confirmation`, `App`, `ResumabilityConfig` |

### 🚀 Production Ready Concepts

You now understand how to build agents that:
- 🌐 **Scale**: Leverage community tools instead of building everything
- ⏳ **Handle Time**: Manage operations that span minutes, hours, or days  
- 🔒 **Ensure Compliance**: Add human oversight to critical operations
- 🔄 **Maintain State**: Resume conversations exactly where they paused

**Start Simple**: Begin with custom tools → Add MCP services → Add long-running as needed

## 🎯 Key Takeaways from the 3 Workflow Demos

The three demos above show the complete image ordering workflow in action:

1. **Demo 1 - Auto-Approval Path**: When requests are small (≤1 picture), approval is automatic
2. **Demo 2 - Approval with YES**: When requests are large (>1 picture), the agent pauses and waits for human decision. Here the human approves.
3. **Demo 3 - Approval with NO**: Same scenario, but the human rejects the large order

**What You Learned:**
- How the `place_get_pic_order` tool with `SUPPORTED_ORIGINS` handles different image sources
- How `tool_context.request_confirmation()` pauses execution waiting for human input
- How the workflow resumes with the same `invocation_id` to continue from where it paused
- How multiple origins (place_get_pic_order, place_get_pig_order, mcp_images, mcp_images_alt) enable flexible sourcing

In [82]:
LARGE_ORDER_THRESHOLD = 1

# Supported image origins with their configurations
SUPPORTED_ORIGINS = {
    "place_get_pic_order": {
        "description": "Standard picture order",
        "image_source": "get_pig_image",  # Function to call for images
        "approval_required_for_bulk": True,
    },
    "place_get_pig_order": {
        "description": "Pig image order (alternative source)",
        "image_source": "get_pig_image",  # Same source, different origin
        "approval_required_for_bulk": True,
    },
    "mcp_images": {
        "description": "MCP server images (Everything server - getTinyImage)",
        "image_source": "mcp_everything",  # Uses Everything MCP server
        "approval_required_for_bulk": False,  # MCP may not need approval
    },
    "mcp_images_alt": {
        "description": "Alternative MCP image source (image-mcp)",
        "image_source": "mcp_image_alt",  # Uses alternative MCP server
        "approval_required_for_bulk": False,  # MCP may not need approval
    },
}


def place_get_pic_order(
    num_pictures: int, origin: str, tool_context: ToolContext
) -> dict:
    """Places an image order. Supports multiple origins (place_get_pic_order, place_get_pig_order, etc.).
    Requires approval if ordering more than 1 picture (LARGE_ORDER_THRESHOLD).

    Args:
        num_pictures: Number of pictures to get
        origin: Pictures origin (place_get_pic_order, place_get_pig_order, mcp_images, etc.)
        tool_context: ADK tool context for confirmation handling

    Returns:
        Dictionary with order status
    """

    # Validate origin
    if origin not in SUPPORTED_ORIGINS:
        return {
            "status": "error",
            "message": f"Unknown origin: {origin}. Supported: {', '.join(SUPPORTED_ORIGINS.keys())}",
        }

    origin_config = SUPPORTED_ORIGINS[origin]

    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤1 picture) auto-approve
    if num_pictures <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_pictures}-AUTO",
            "num_pictures": num_pictures,
            "origin": origin,
            "origin_description": origin_config["description"],
            "image_source": origin_config["image_source"],
            "message": f"Order auto-approved: {num_pictures} pictures from {origin}",
        }

    # Check if this origin requires bulk approval
    if not origin_config["approval_required_for_bulk"]:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_pictures}-AUTO-BULK",
            "num_pictures": num_pictures,
            "origin": origin,
            "origin_description": origin_config["description"],
            "image_source": origin_config["image_source"],
            "message": f"Bulk order auto-approved from {origin}: {num_pictures} pictures",
        }

    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_pictures} pictures from {origin} ({origin_config['description']}). Do you want to approve?",
            payload={"num_pictures": num_pictures, "origin": origin},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_pictures} pictures from {origin} requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_pictures}-HUMAN",
            "num_pictures": num_pictures,
            "origin": origin,
            "origin_description": origin_config["description"],
            "image_source": origin_config["image_source"],
            "message": f"Order approved: {num_pictures} pictures from {origin}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_pictures} pictures from {origin}",
        }


print("✅ Long-running functions created with support for multiple origins!")

✅ Long-running functions created with support for multiple origins!


In [83]:

# Create a pig_image_server tool that fetches the pig image from httpbin.org
import requests
import base64

def get_pig_image() -> dict:
    """Fetches a pig image from a public source, with fallback to embedded pig image.
    
    Returns:
        Dictionary with image data in base64 format
    """
    try:
        response = requests.get("https://httpbin.org/image/png", timeout=3, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            img_base64 = base64.b64encode(response.content).decode('utf-8')
            return {
                "status": "success",
                "type": "image",
                "data": img_base64,
                "source": "httpbin.org/image/png"
            }
    except Exception as e:
        pass
    
    # Fallback: Use embedded pig image (16x16 blue/red checkerboard that looks like the pig emoji)
    # This is a valid PNG that will display reliably
    fallback_pig_png = base64.b64decode(
        "iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAIAAAB7QOjdAAAAH0lEQVR4nGNkYGD4DwQYGBiYGBiZmBkYmBj+P2BkZPjPwAgAOsgJB5SqpBYAAAAASUVORK5CYII="
    )
    img_base64 = base64.b64encode(fallback_pig_png).decode('utf-8')
    return {
        "status": "success",
        "type": "image",
        "data": img_base64,
        "source": "fallback_pig_image"
    }

pig_image_tool = FunctionTool(func=get_pig_image)
print("✅ Pig image tool created!")


✅ Pig image tool created!


In [84]:
# Create image agent with MCP integration and pig image tool
# Build tools list based on what's available
tools_list = [FunctionTool(func=place_get_pic_order), pig_image_tool]

# Check if mcp_image_server is available
try:
    mcp_available = mcp_image_server is not None
    print(f"[INFO] mcp_image_server available: {mcp_available}")
except:
    mcp_available = False
    print("[INFO] mcp_image_server NOT available")

if mcp_available:
    tools_list.append(mcp_image_server)
    print("[INFO] Added mcp_image_server to agent tools")
else:
    print("[WARNING] mcp_image_server not available")

# Also add mcp_image_alternative if available
try:
    mcp_alt_available = mcp_image_alternative is not None
    print(f"[INFO] mcp_image_alternative available: {mcp_alt_available}")
except:
    mcp_alt_available = False
    print("[INFO] mcp_image_alternative NOT available")

if mcp_alt_available:
    tools_list.append(mcp_image_alternative)
    print("[INFO] Added mcp_image_alternative to agent tools")
else:
    print("[INFO] mcp_image_alternative not available - using existing tools only")

image_agent = LlmAgent(
    name="image_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are an image provider coordinator.

When users request images, support these origins:
- "place_get_pic_order": Standard picture order
- "place_get_pig_order": Alternative pig image order  
- "mcp_images": MCP Everything server images
- "mcp_images_alt": Alternative MCP image source

Workflow:
1. First, use the place_get_pic_order tool with the number of pictures and specify the origin.
   - If not specified, use "place_get_pic_order" as default
2. After place_get_pic_order returns:
   - If status is 'rejected': Inform user the order was rejected
   - If status is 'pending': Inform user that approval is required and wait for response
   - If status is 'approved': Request images based on the image_source from the response
3. Display all images obtained (or emoji 🐷 as fallback if images unavailable)
4. Provide a final summary including:
   - Order status (approved/rejected)
   - Order ID (if available)
   - Number of pictures and origin
   - All retrieved pictures (with emoji fallback 🐷 if needed)

IMPORTANT: When order is approved, call image retrieval ONCE for EACH picture (requested one at a time, not in a loop):
- For 1 picture: Call once
- For 2 pictures: Call twice (separately)
- For 3 pictures: Call three times (separately)

If images cannot be retrieved, use emoji fallback: 🐷 (one per picture).
Do not try to call tools with multiple parameters - each call should be independent.
""",
    tools=tools_list,
)

print("✅ Image Agent created!")


[INFO] mcp_image_server available: True
[INFO] Added mcp_image_server to agent tools
[INFO] mcp_image_alternative NOT available
[INFO] mcp_image_alternative not available - using existing tools only
✅ Image Agent created!


In [85]:

# DIAGNOSTIC: Check if mcp_image_server is available
print("[DIAGNOSTIC] Checking mcp_image_server...")
try:
    print(f"  mcp_image_server type: {type(mcp_image_server)}")
    print(f"  mcp_image_server: {mcp_image_server}")
    print("  ✅ mcp_image_server is available")
except NameError as e:
    print(f"  ❌ ERROR: mcp_image_server is NOT defined: {e}")
    print("  This means the MCP server setup failed earlier in the notebook")


[DIAGNOSTIC] Checking mcp_image_server...
  mcp_image_server type: <class 'google.adk.tools.mcp_tool.mcp_toolset.McpToolset'>
  mcp_image_server: <google.adk.tools.mcp_tool.mcp_toolset.McpToolset object at 0x123026230>
  ✅ mcp_image_server is available


In [86]:
# Alternative MCP server for images: image-mcp
# This server provides additional image generation/retrieval capabilities
# You can use this as an alternative source for "mcp_images" origin

try:
    # Try to set up image-mcp server
    # Note: This requires the @modelcontextprotocol/image-mcp package to be installed
    # If not available, it will be skipped gracefully
    mcp_image_alternative = McpToolset(
        connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command="npx",  # Run MCP server via npx
                args=[
                    "-y",  # Argument for npx to auto-confirm install
                    "@modelcontextprotocol/server-image",  # Image MCP server
                    # Note: Adjust based on actual image-mcp package name
                ],
                # Can filter specific tools if needed
                # tool_filter=["generateImage", "getImage"],  # Optional
            ),
            timeout=30,
        )
    )
    print("✅ Image-MCP Tool created (alternative source)")
except Exception as e:
    print(f"⚠️ Image-MCP not available: {e}")
    print("   Using Everything server as primary MCP source")
    mcp_image_alternative = None


✅ Image-MCP Tool created (alternative source)


In [87]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
get_pic_app = App(
    name="get_pic_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/var/folders/wl/2rc_1jsx16320v3qt__jc90r0000gn/T/ipykernel_84954/472175699.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [89]:
# from google.adk.runners import InMemoryRunner
# 
# runner = InMemoryRunner(agent=image_agent)


session_service = InMemorySessionService()

# Create runner with the resumable app
get_pic_runner = Runner(
    app=get_pic_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


In [90]:
def print_agent_response(events):
    """Print agent's text responses from events."""
#    for event in events:
#        if event.content and event.content.parts:
#            for part in event.content.parts:
#                if part.text:
#                    print(f"Agent > {part.text}")

    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")
                if hasattr(part, "function_response") and part.function_response:
                    for item in part.function_response.response.get("content", []):
                        if item.get("type") == "image":
                            display(IPImage(data=base64.b64decode(item["data"])))


In [91]:
from IPython.display import display, Image as IPImage, HTML
import base64


async def run_get_pic_workflow(query: str, auto_approve: bool = True, debug: bool = False):
    """Runs a get picture workflow with approval handling.

    Args:
        query: User's picture request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
        debug: Print detailed event information
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="get_pic_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # STEP 1: Send initial request to the Agent
    async for event in get_pic_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)
        if debug:
            print(f"[DEBUG] Event received: author={event.author}, invocation_id={event.invocation_id}")
            if event.content and event.content.parts:
                for i, part in enumerate(event.content.parts):
                    if hasattr(part, 'text') and part.text:
                        print(f"  Part {i} (text): {part.text[:100]}")
                    if hasattr(part, 'function_call') and part.function_call:
                        print(f"  Part {i} (function_call): name={part.function_call.name}, id={part.function_call.id}")
                    if hasattr(part, 'function_response') and part.function_response:
                        print(f"  Part {i} (function_response): {part.function_response}")

    # STEP 2: Check if approval is needed
    approval_info = check_for_approval(events)
    
    print(f"\n[RESULT] approval_info = {approval_info}")
    print(f"[RESULT] Total events received: {len(events)}")

    # STEP 3: Handle approval if needed
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume with approval decision
        resume_events = []
        image_count = 0
        async for event in get_pic_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),
            invocation_id=approval_info["invocation_id"],
        ):
            resume_events.append(event)
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
                    if hasattr(part, "function_response") and part.function_response:
                        # Process function responses to extract images
                        response_data = part.function_response.response
                        if debug:
                            print(f"[DEBUG] Function response: {response_data}")
                        
                        image_found = False
                        # Check for image content in various formats
                        if isinstance(response_data, dict):
                            # Try to get content array
                            content_list = response_data.get("content", [])
                            if isinstance(content_list, list):
                                for item in content_list:
                                    if isinstance(item, dict) and item.get("type") == "image":
                                        try:
                                            img_data = item.get("data", "")
                                            if img_data:
                                                display(IPImage(data=base64.b64decode(img_data)))
                                                image_found = True
                                                image_count += 1
                                        except Exception as e:
                                            if debug:
                                                print(f"[DEBUG] Error displaying image: {e}")
                            # Also check if response_data has direct image data
                            elif response_data.get("type") == "image":
                                try:
                                    img_data = response_data.get("data", "")
                                    if img_data:
                                        display(IPImage(data=base64.b64decode(img_data)))
                                        image_found = True
                                        image_count += 1
                                except Exception as e:
                                    if debug:
                                        print(f"[DEBUG] Error displaying image: {e}")
                        
                        # Emoji fallback if no image found
                        if not image_found and isinstance(response_data, dict) and response_data.get("status") == "success":
                            display(HTML("<span style='font-size: 48px;'>🐷</span>"))
                            image_count += 1

    else:
        # PATH B: No approval needed - order completed immediately
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")


✅ Workflow function ready


## 🎬 Section 5: Running the 3 Image Ordering Demos

Now let's test the image ordering system with three different scenarios. Each demo showcases a different approval workflow:

**Demo 1: Small Order (Auto-Approve)**
- Single picture request → Auto-approved (no human needed)
- Shows the simple case where ordering is straightforward

**Demo 2: Bulk Order with Approval**
- Multiple pictures requested → Requires approval → Human says YES ✅
- Shows the approval workflow when order is large

**Demo 3: Bulk Order with Rejection**
- Multiple pictures requested → Requires approval → Human says NO ❌
- Shows what happens when approval is denied

**Note:** You may see warnings like `Warning: there are non-text parts in the response: ['function_call']` - this is normal. It means the agent is calling tools.

## 🚀 Run the Demos Now!

Execute the cell below to see all three scenarios in action. Watch how:
- **Demo 1**: Small order → Instant approval
- **Demo 2**: Large order → Agent pauses → Human approves → Workflow resumes
- **Demo 3**: Large order → Agent pauses → Human rejects → Workflow resumes

Each demo will print detailed output showing the complete workflow cycle.

In [92]:
# Demo 1: Small order - Auto-approved
print("DEMO 1: Single picture request (should auto-approve)")
await run_get_pic_workflow("Get 1 picture from place_get_pic_order", debug=True)

# Demo 2: Workflow simulates human decision: APPROVE ✅
print("DEMO 2: Multiple pictures with APPROVAL")
await run_get_pic_workflow("Get 2 pictures from place_get_pig_order", auto_approve=True, debug=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
print("DEMO 3: Multiple pictures with REJECTION")
await run_get_pic_workflow("Get 3 pictures from place_get_pic_order", auto_approve=False, debug=True)


DEMO 1: Single picture request (should auto-approve)

User > Get 1 picture from place_get_pic_order



[DEBUG] Event received: author=image_agent, invocation_id=e-01f39962-1967-47dc-8ba6-d3efb6a07750
  Part 0 (function_call): name=place_get_pic_order, id=adk-dc20326f-adc3-43b3-a92d-20f15ddfc07b
[DEBUG] Event received: author=image_agent, invocation_id=e-01f39962-1967-47dc-8ba6-d3efb6a07750
  Part 0 (function_response): will_continue=None scheduling=None parts=None id='adk-dc20326f-adc3-43b3-a92d-20f15ddfc07b' name='place_get_pic_order' response={'status': 'approved', 'order_id': 'ORD-1-AUTO', 'num_pictures': 1, 'origin': 'place_get_pic_order', 'origin_description': 'Standard picture order', 'image_source': 'get_pig_image', 'message': 'Order auto-approved: 1 pictures from place_get_pic_order'}


[DEBUG] Event received: author=image_agent, invocation_id=e-01f39962-1967-47dc-8ba6-d3efb6a07750
  Part 0 (function_call): name=get_pig_image, id=adk-30c0295b-a955-4272-90f2-f48c8a74d3b6
[DEBUG] Event received: author=image_agent, invocation_id=e-01f39962-1967-47dc-8ba6-d3efb6a07750
  Part 0 (function_response): will_continue=None scheduling=None parts=None id='adk-30c0295b-a955-4272-90f2-f48c8a74d3b6' name='get_pig_image' response={'status': 'success', 'type': 'image', 'data': 'iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAIAAAB7QOjdAAAAH0lEQVR4nGNkYGD4DwQYGBiYGBiZmBkYmBj+P2BkZPjPwAgAOsgJB5SqpBYAAAAASUVORK5CYII=', 'source': 'fallback_pig_image'}
[DEBUG] Event received: author=image_agent, invocation_id=e-01f39962-1967-47dc-8ba6-d3efb6a07750
  Part 0 (function_response): will_continue=None scheduling=None parts=None id='adk-30c0295b-a955-4272-90f2-f48c8a74d3b6' name='get_pig_image' response={'status': 'success', 'type': 'image', 'data': 'iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAIAAAB7QOjdAAAAH0lEQVR4nGNk

/usr/local/Cellar/ipython/8.5.0/libexec/lib/python3.10/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/usr/local/Cellar/ipython/8.5.0/libexec/lib/python3.10/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()
/usr/local/Cellar/ipython/8.5.0/libexec/lib/python3.10/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/usr/local/Cellar/ipython

[DEBUG] Event received: author=image_agent, invocation_id=e-3bb093f1-7c5f-4e82-97aa-e4021245795f
  Part 0 (function_call): name=place_get_pic_order, id=adk-a4ed1c7a-3adc-4dd0-bbcf-c3ba516fb085
[DEBUG] Event received: author=image_agent, invocation_id=e-3bb093f1-7c5f-4e82-97aa-e4021245795f
  Part 0 (function_call): name=adk_request_confirmation, id=adk-9b829c0a-ad33-4479-82d5-4aaadeaddd2e
[DEBUG] Event received: author=image_agent, invocation_id=e-3bb093f1-7c5f-4e82-97aa-e4021245795f
  Part 0 (function_response): will_continue=None scheduling=None parts=None id='adk-a4ed1c7a-3adc-4dd0-bbcf-c3ba516fb085' name='place_get_pic_order' response={'status': 'pending', 'message': 'Order for 2 pictures from place_get_pig_order requires approval'}

[RESULT] approval_info = {'approval_id': 'adk-9b829c0a-ad33-4479-82d5-4aaadeaddd2e', 'invocation_id': 'e-3bb093f1-7c5f-4e82-97aa-e4021245795f'}
[RESULT] Total events received: 3
⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

[DEBUG] Function r

[DEBUG] Event received: author=image_agent, invocation_id=e-8a340b39-f749-48da-a368-9d7d46a89d5c
  Part 0 (function_call): name=place_get_pic_order, id=adk-b000c13c-83e4-45e1-94ab-12d0a0cc53d3
[DEBUG] Event received: author=image_agent, invocation_id=e-8a340b39-f749-48da-a368-9d7d46a89d5c
  Part 0 (function_call): name=adk_request_confirmation, id=adk-9ae7cee6-3c30-4544-8018-e60143ef896c
[DEBUG] Event received: author=image_agent, invocation_id=e-8a340b39-f749-48da-a368-9d7d46a89d5c
  Part 0 (function_response): will_continue=None scheduling=None parts=None id='adk-b000c13c-83e4-45e1-94ab-12d0a0cc53d3' name='place_get_pic_order' response={'status': 'pending', 'message': 'Order for 3 pictures from place_get_pic_order requires approval'}

[RESULT] approval_info = {'approval_id': 'adk-9ae7cee6-3c30-4544-8018-e60143ef896c', 'invocation_id': 'e-8a340b39-f749-48da-a368-9d7d46a89d5c'}
[RESULT] Total events received: 3
⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

[DEBUG] Function re

In [93]:

# Quick test: Verify the pig_image_tool is working
print("Testing pig_image_tool...")
pig_result = get_pig_image()
print(f"✅ Pig image tool returned: {pig_result['status']}")
print(f"   Source: {pig_result['source']}")
print(f"   Image data length: {len(pig_result['data'])} characters")

# Display it
if pig_result['status'] == 'success':
    display(IPImage(data=base64.b64decode(pig_result['data'])))
    print("✅ Pig image displayed successfully!")


Testing pig_image_tool...
✅ Pig image tool returned: success
   Source: fallback_pig_image
   Image data length: 124 characters
✅ Pig image tool returned: success
   Source: fallback_pig_image
   Image data length: 124 characters


✅ Pig image displayed successfully!


In [94]:

# Test: Fetch a tiny image from a public resource
import requests
from IPython.display import display, Image as IPImage
import base64

print("Fetching a tiny image from public resources...")

# Try multiple public image URLs
urls = [
    ("Placeholder.com 50x50", "https://via.placeholder.com/50"),
    ("DuckDuckGo logo", "https://duckduckgo.com/assets/favicon-96x96.png"),
    ("Httpbin image", "https://httpbin.org/image/png"),
]

image_fetched = False

for name, url in urls:
    try:
        print(f"\n📍 Trying: {name}")
        response = requests.get(url, timeout=5, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            # Display the image
            display(IPImage(data=response.content))
            print(f"✅ Successfully fetched image from: {url}")
            print(f"   Image size: {len(response.content)} bytes")
            image_fetched = True
            break
        else:
            print(f"❌ HTTP {response.status_code}")
    except Exception as e:
        print(f"❌ Error: {e}")

if not image_fetched:
    # Fallback: Create a minimal valid PNG (16x16 colorful pixel grid)
    print("\n🎨 Fallback: Creating a test image...")
    # 16x16 PNG with a simple pattern - red/blue checkerboard
    fallback_png = base64.b64decode(
        "iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAIAAAB7QOjdAAAAH0lEQVR4nGNkYGD4DwQYGBiYGBiZmBkYmBj+P2BkZPjPwAgAOsgJB5SqpBYAAAAASUVORK5CYII="
    )
    display(IPImage(data=fallback_png))
    print("✅ Displayed 16x16 test image (blue/red checkerboard)")


Fetching a tiny image from public resources...

📍 Trying: Placeholder.com 50x50
❌ Error: HTTPSConnectionPool(host='via.placeholder.com', port=443): Max retries exceeded with url: /50 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x122de40a0>: Failed to resolve 'via.placeholder.com' ([Errno 8] nodename nor servname provided, or not known)"))

📍 Trying: DuckDuckGo logo
❌ HTTP 410

📍 Trying: Httpbin image
❌ Error: HTTPSConnectionPool(host='via.placeholder.com', port=443): Max retries exceeded with url: /50 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x122de40a0>: Failed to resolve 'via.placeholder.com' ([Errno 8] nodename nor servname provided, or not known)"))

📍 Trying: DuckDuckGo logo
❌ HTTP 410

📍 Trying: Httpbin image
❌ HTTP 503

🎨 Fallback: Creating a test image...
❌ HTTP 503

🎨 Fallback: Creating a test image...


✅ Displayed 16x16 test image (blue/red checkerboard)


---

## 🎉 Congratulations! You've Learned Agent Patterns and Best Practices

You've successfully learned how to build agents that handle complex, real-world workflows integrating external systems and spanning time.

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

- [ADK Documentation](https://google.github.io/adk-docs/)
- [MCP Tools Documentation](https://google.github.io/adk-docs/tools/mcp-tools/)
- [Long-Running Operations Guide](https://google.github.io/adk-docs/tools/function-tools/)
- [Model Context Protocol Specification](https://spec.modelcontextprotocol.io/)
- [The `App` and `Runner`](https://google.github.io/adk-docs/runtime/)

### 🎯 Next Steps

You've built the foundation for production-ready agent systems. Ready for the next challenge?

Continue to **Day 3** to learn about **State and Memory Management**!

---

<div align="center">
  <table>
    <tr>
      <th style="text-align:center">Authors</th>
    </tr>
    <tr>
      <td style="text-align:center"><a href="https://www.linkedin.com/in/laxmi-harikumar/">Laxmi Harikumar</a></td>
    </tr>
  </table>
</div>